In [1]:
import pya

In [123]:
def merge_polygons(polys):
    """Merges the shapes in polys into one polygon. Overlaps will be merged into one.
    
    @param list polys: list of the polygons that should be merged.

    @return poly.Region: region containing the merged polygon
    """
    union = pya.Region()
    for poly in polys:
        union += pya.Region(poly)
    merged = union.merge()
    return(merged)


def merge_regions(regions):
    """Merges the regions in regions into one region. Overlaps will be merged into one?
    
    @param list regions: list of the regions that should be merged.

    @return Region: region containing the merged regions
    """
    union = pya.Region()
    for region in regions:
        union += region
    merged = union.merge()
    return(merged)


def create_box(center, width, height):
    """Creates a box.
    
    @param list center: x and y coordinate of the center of the box

    @param float width: width (x) of the box

    @param float height: height (y) of the box

    @return Box: Box object
    """
    box = pya.Box.new(pya.Point(center[0]-width/2,center[1]-height/2),pya.Point(center[0]+width/2,center[1]+height/2))
    return(box)


def create_text(text,gridsize,height,posi):
    """Creates a text object at the specified location.
    
    @param str text: ext to display

    @param float gridsize: size of the used grid (layout.dbu)

    @param float height: height of the text

    @param list posi: x and y coordinate of the box (lower left corner)
    """
    scale = height * 10/7 * 1e-3 # rescaling so size is in dbu
    region = pya.TextGenerator.default_generator().text(text,gridsize/scale)
    region.move(int(posi[0]), int(posi[1]))
    return(region)


def create_marker(center,width):
    """Creates a marker that is centered at center and width wide and tall.
    """
    boxlength = 0.3*width
    boxwidth = 0.1*width
    connectorwidth = 0.025*width
    arms_offset = 0.5*width-0.5*boxlength

    elements = []
    elements.append(create_box(center, width, connectorwidth)) # connector_vertical
    elements.append(create_box(center, connectorwidth, width)) # connector_horizontal
    elements.append(create_box([center[0],center[1]+arms_offset],boxwidth,boxlength)) # upper arm
    elements.append(create_box([center[0],center[1]-arms_offset],boxwidth,boxlength)) # lower arm
    elements.append(create_box([center[0]-arms_offset,center[1]],boxlength,boxwidth)) # left arm
    elements.append(create_box([center[0]+arms_offset,center[1]],boxlength,boxwidth)) # right arm

    marker = merge_polygons(elements)

    return(marker)


def create_marker_matrix(center_marker1,width_marker,repetitions,offsets):
    """ Creates an array of markers
    
    @param list center_marker1: list of floats that gives the position (x and y) of the marker in the lower left corner of the array

    @param float width_marker: size of the marker

    @param list repetitions: list of ints with the number of repetitions in x and y direction

    @param list offsets: list of floats that gives the offset between markers in x and y direction

    @return Region: region that contains the markers
    """

    markers = []
    center = [0,0] # initialize center
    for i in range(repetitions[0]):
        center[0] = center_marker1[0] + i*offsets[0]
        for j in range(repetitions[1]):
            center[1] = center_marker1[1] + j*offsets[1]
            marker = create_marker(center,width_marker)
            markers.append(marker)
    matrix = merge_regions(markers)
    
    return(matrix)


def create_numbers_for_marker(center,gridsize,crosswidth,numbers):
    """Creates 4 numbers in a square.

    If set up correctly, numbers should be in quadrants of the corresponding marker/cross.

    @param list center: x and y coordinate of the crosses center

    @param float gridsize: size of hte used grid (layout.dbu)

    @param float crosswidth: width of the corresponding cross

    @param list numbers: numbers as string that should be placed in the quadrants.
        The order is a follows: [top left, top right, lower left, lower right]
    """
    textheight = 0.3*crosswidth
    elements = []
    elements.append(create_text(numbers[0],gridsize,textheight,[center[0]-0.4*crosswidth,center[1]+0.1*crosswidth])) # top left
    elements.append(create_text(numbers[1],gridsize,textheight,[center[0]+0.1*crosswidth,center[1]+0.1*crosswidth])) # top right
    elements.append(create_text(numbers[2],gridsize,textheight,[center[0]-0.4*crosswidth,center[1]-0.4*crosswidth])) # bottom left
    elements.append(create_text(numbers[3],gridsize,textheight,[center[0]+0.1*crosswidth,center[1]-0.4*crosswidth])) # bottom right

    numbers = merge_regions(elements)

    return numbers


def create_number_matrix(center_marker1,gridsize,width_marker,repetitions,offsets):
    """ Creates an array of numbers.
    
    @param list center_marker1: list of floats that gives the position (x and y) of the corresponding marker in the lower left corner of the array

    @param float width_marker: size of the corresponding marker

    @param list repetitions: list of ints with the number of repetitions in x and y direction

    @param list offsets: list of floats that gives the offset between corresponding markers in x and y direction

    @return Region: region that contains the numbers
    """
    elements = []
    center = [0,0] # initialize center
    for i in range(repetitions[0]):
        center[0] = center_marker1[0] + i*offsets[0]
        num_x = f'{i:02}'
        for j in range(repetitions[1]):
            center[1] = center_marker1[1] + j*offsets[1]
            num_y = f'{j:02}'
            numbers = [num_x[:-1],num_x[-1],num_y[:-1],num_y[-1]] # takes last number in right quadrants, rest (usually only 1) in left quadrants
            nums = create_numbers_for_marker(center,gridsize,width_marker,numbers)
            elements.append(nums)
    matrix = merge_regions(elements)

    return matrix

In [125]:
## create new layout
layout = pya.Layout()
layout.dbu = 1e-3 # data base unit in um --> each number we put is now in nm

top = layout.create_cell("TOP")
l1 = layout.layer(1, 0)
l2 = layout.layer(2,0)

## create structures
center_marker1 = [5000,5000]
width_marker = 10000
repetitions = [3,5]
offsets = [30000,20000]
marker_matrix = create_marker_matrix(center_marker1=center_marker1,
                                        width_marker=width_marker,
                                        repetitions=repetitions, 
                                        offsets=offsets)
number_matrix = create_number_matrix(center_marker1=center_marker1,
                                        gridsize=layout.dbu,
                                        width_marker=width_marker,
                                        repetitions=repetitions,
                                        offsets=offsets)

## place structures
top.shapes(l1).insert(marker_matrix)
top.shapes(l2).insert(number_matrix)

## save in file
layout.write('t.gds')